In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

CCAL version 0.9.1 @ /home/kwatme/github/ccal/ccal/__init__.py


In [3]:
gps_map = ccal.load_gps_map(PATH["gps_map.pickle.gz"])

In [4]:
feature_data = SETTING["feature_data"]

if feature_data is None:

    feature_data = {}

for data_name, data_dict in feature_data.items():

    print(data_name)

    data_dict["df"] = pd.read_table(data_dict["file_path"], index_col=0)

In [5]:
sample_data = SETTING["sample_data"]

if sample_data is None:

    sample_data = {}

for data_name, data_dict in sample_data.items():

    print(data_name)

    data_dict["df"] = pd.read_table(data_dict["file_path"], index_col=0)

Tissue


In [6]:
for element_type, w_or_h, elements, element_labels, data_dicts in (
    ("feature", "w", gps_map.w_elements, gps_map.w_element_labels, feature_data),
    ("sample", "h", gps_map.h_elements, gps_map.h_element_labels, sample_data),
):

    for data_name, data_dict in data_dicts.items():

        if "indices" not in data_dict:

            continue

        df = data_dict["df"]

        if data_dict["indices"] == "all":

            indices = df.index

        else:

            indices = data_dict["indices"]

        for index, element_value in df.loc[indices, df.columns & elements].iterrows():

            if element_value.isna().all():

                continue

            if data_dict["type"] == "binary" and (element_value == 0).all():

                continue

            if data_dict["type"] == "categorical":

                colorscale = ccal.make_colorscale(
                    n_category=element_value.dropna().unique().size, plot=False
                )

            else:

                colorscale = None

            title = "{} GPS Map<br>{}: {}".format(
                SETTING["{}_alias".format(element_type)], data_name, index
            )

            file_name = ccal.make_file_name_from_str(
                "{}_{}.html".format(data_name, index)
            )

            if element_labels is None:

                gps_map_directory_path = PATH["{}|gps_map/".format(w_or_h)]

            else:

                gps_map_directory_path = PATH["{}|hcc|gps_map/".format(w_or_h)]

            html_file_path = "{}/{}".format(gps_map_directory_path, file_name)

            if SETTING["plotly_directory_path"] is None or element_labels is None:

                plotly_html_file_path = None

            else:

                plotly_html_file_path = "{}/{}".format(
                    PATH["plotly|{}_gps_map/".format(w_or_h)], file_name
                )

            gps_map.plot_gps_map(
                w_or_h,
                annotation_x_element=element_value.to_frame().T,
                annotation_std_maxs=(SETTING["plot_std"],),
                annotation_types=(data_dict["type"],),
                annotation_colorscale=colorscale,
                element_marker_size=SETTING[
                    "gps_map_{}_element_marker_size".format(w_or_h)
                ],
                title=title,
                html_file_path=html_file_path,
                plotly_html_file_path=plotly_html_file_path,
            )

            if element_labels is None:

                continue

            label_element_value = element_value.groupby(by=element_labels)

            names = tuple(
                "{} Cluster {}".format(SETTING["{}_alias".format(element_type)], label)
                for label in label_element_value.groups.keys()
            )

            html_file_path = "{}/{}".format(
                PATH["{}|hcc|comparison/".format(w_or_h)], file_name
            )

            xaxis_title = "{} Cluster".format(SETTING["{}_alias".format(element_type)])

            if data_dict["type"] in ("continuous", "categorical"):

                ccal.plot_violin_or_box(
                    (element_value_ for label, element_value_ in label_element_value),
                    names=names,
                    violin_or_box="box",
                    title=title,
                    xaxis_title=xaxis_title,
                    yaxis_title="Value",
                    html_file_path=html_file_path,
                )

            elif data_dict["type"] == "binary":

                ccal.plot_bar(
                    ((name,) for name in names),
                    (
                        (label_density,)
                        for label_density in label_element_value.sum()
                        / label_element_value.size()
                    ),
                    names=names,
                    title=title,
                    xaxis_title=xaxis_title,
                    yaxis_title="Density",
                    html_file_path=html_file_path,
                )

file:///media/kwatme/CarrotCake/project/model_and_infer.gtex_v7_ribosomal_rna_in_blood_and_brain/output/signal/raw/nmf/3/h/gps_map/Tissue_Brain_-_Amygdala.html


file:///media/kwatme/CarrotCake/project/model_and_infer.gtex_v7_ribosomal_rna_in_blood_and_brain/output/signal/raw/nmf/3/h/gps_map/Tissue_Brain_-_Anterior_cingulate_cortex_BA24.html


file:///media/kwatme/CarrotCake/project/model_and_infer.gtex_v7_ribosomal_rna_in_blood_and_brain/output/signal/raw/nmf/3/h/gps_map/Tissue_Brain_-_Caudate_basal_ganglia.html


file:///media/kwatme/CarrotCake/project/model_and_infer.gtex_v7_ribosomal_rna_in_blood_and_brain/output/signal/raw/nmf/3/h/gps_map/Tissue_Brain_-_Cerebellar_Hemisphere.html


file:///media/kwatme/CarrotCake/project/model_and_infer.gtex_v7_ribosomal_rna_in_blood_and_brain/output/signal/raw/nmf/3/h/gps_map/Tissue_Brain_-_Cerebellum.html


...

file:///media/kwatme/CarrotCake/project/model_and_infer.gtex_v7_ribosomal_rna_in_blood_and_brain/output/signal/raw/nmf/3/h/gps_map/Tissue_Brain_-_Hypothalamus.html


file:///media/kwatme/CarrotCake/project/model_and_infer.gtex_v7_ribosomal_rna_in_blood_and_brain/output/signal/raw/nmf/3/h/gps_map/Tissue_Brain_-_Putamen_basal_ganglia.html


file:///media/kwatme/CarrotCake/project/model_and_infer.gtex_v7_ribosomal_rna_in_blood_and_brain/output/signal/raw/nmf/3/h/gps_map/Tissue_Brain_-_Spinal_cord_cervical_c-1.html


file:///media/kwatme/CarrotCake/project/model_and_infer.gtex_v7_ribosomal_rna_in_blood_and_brain/output/signal/raw/nmf/3/h/gps_map/Tissue_Brain_-_Substantia_nigra.html


file:///media/kwatme/CarrotCake/project/model_and_infer.gtex_v7_ribosomal_rna_in_blood_and_brain/output/signal/raw/nmf/3/h/gps_map/Tissue_Whole_Blood.html
